In [1]:
#!/usr/bin/env python
################################################################
#  Copyright (C) 2015 OpenEye Scientific Software, Inc.
################################################################
from __future__ import print_function
import os, sys
import numpy as np

In [2]:
import openeye.oechem as oechem

In [3]:
from simtk import unit, openmm
from simtk.openmm import app
from simtk.openmm import XmlSerializer
from sys import stdout

import parmed
import base64
import pickle

import json
import plmd

In [4]:
inFileName = 'input/WNE6S-warmup.oeb.gz'
ifs = oechem.oemolistream(inFileName)
mol = oechem.OEMol()
oechem.OEReadMolecule(ifs, mol)

True

In [5]:
structTag = 'structure'
if structTag in mol.GetData().keys():
    print('mol GetData finds tag:', structTag)
    encoded_struct = mol.GetData( structTag)

mol GetData finds tag: structure


In [6]:
decoded_structure = base64.b64decode( encoded_struct)
struct_dict = pickle.loads(decoded_structure)
struct = parmed.structure.Structure()
struct.__setstate__(struct_dict)
#help( struct)

In [7]:
topology = struct.topology
initPositions = struct.positions

In [8]:
print(initPositions[0])

(33.43, 11.279999999999999, 44.490000000000009) A


In [9]:
systemTag = 'system'
if systemTag in mol.GetData().keys():
    print('mol GetData finds tag:', systemTag)
    serialized_system = mol.GetData( systemTag)
    system = openmm.XmlSerializer.deserialize(serialized_system)

mol GetData finds tag: system


In [10]:
if 'state' in mol.GetData().keys():
    print('mol GetData finds tag:', 'state')
    mol.GetData(oechem.OEGetTag('state'))
    serialized_state = mol.GetData(oechem.OEGetTag('state'))
    state = openmm.XmlSerializer.deserialize( serialized_state )

mol GetData finds tag: state


In [11]:
pos = state.getPositions()
print( len( pos))
print( pos[:3])
#veloc = state.getVelocities()

32357
[(3.3569929599761963, 1.1250951290130615, 4.444279670715332), (3.36295747756958, 1.2007523775100708, 4.317758560180664), (3.222506284713745, 1.236323595046997, 4.273021697998047)] nm


In [12]:
dataTagForOpenMM_velocities = 'OpenMM_Velocities_json'
if mol.HasData(oechem.OEGetTag( dataTagForOpenMM_velocities)):
    print('reading velocities from mol Generic Data with tag:', dataTagForOpenMM_velocities)
    velocities = json.loads( mol.GetStringData( 'OpenMM_Velocities_json'))*(unit.nanometer/unit.picosecond)

reading velocities from mol Generic Data with tag: OpenMM_Velocities_json


In [13]:
print( len( velocities))
print( velocities[:3])

32357
[[0.03618885949254036, -0.557714581489563, 1.0361324548721313], [-0.3941652178764343, 0.4534037113189697, 0.5156360268592834], [0.06389316916465759, -0.3627958297729492, 0.11415818333625793]] nm/ps


In [14]:
dataTagForOpenMM_PLmask = 'OpenMM_PLmaskDict_json'
if mol.HasData(oechem.OEGetTag( dataTagForOpenMM_PLmask)):
    print('reading OpenMM PLmask from mol Generic Data with tag:', dataTagForOpenMM_PLmask)
    PLMask = json.loads(mol.GetStringData( dataTagForOpenMM_PLmask))

reading OpenMM PLmask from mol Generic Data with tag: OpenMM_PLmaskDict_json


In [15]:
OpenMMstuff = { 'idtag': 'WNE6S',
                'system': system,
                'topology': topology,
                'positions': initPositions,
                'velocities': velocities,
                'state': state,
                'PLmask': PLMask }

In [16]:
argsDict = {'picosec': 5.0, 'temperature': 300.0, 'snapFreq': 0, 'restraintWt': 2.0}

In [17]:
equilState = plmd.RestrEquil( OpenMMstuff, argsDict)

RestrEquil: Warm up for 5 picoseconds with 2.0 kcal/mol/ang^2 restraints on all non-water non-Hydrogens
RestrEquil: building system and simulation objects
velCheck:  32357
velCheck:  [(0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0)] nm/ps
velocities:  32357
velocities:  [[0.03618885949254036, -0.557714581489563, 1.0361324548721313], [-0.3941652178764343, 0.4534037113189697, 0.5156360268592834], [0.06389316916465759, -0.3627958297729492, 0.11415818333625793]] nm/ps


2017-02-20 14:13:24 1487625204.83 | 0.80 CPU sec: RestrEquil: system and simulation objects created and initialized


velCheck:  32357
velCheck:  [(0.03618885949254036, -0.557714581489563, 1.0361324548721313), (-0.3941652178764343, 0.4534037113189697, 0.5156360268592834), (0.06389316916465759, -0.3627958297729492, 0.11415818333625793)] nm/ps
RestrEquil: Computations will be done on platform: OpenCL
reporting frequency is 50
totalTimeSteps is 2500
RestrEquil: starting MD for 5.000 picoseconds (2500 timesteps)
#"Step","Time (ps)","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)","Density (g/mL)"
50,0.10000000000000007,-483861.14604205056,46381.74434831782,-437479.4016937327,146.88596063585936,0.9611142796970737
100,0.20000000000000015,-477746.44661901286,50779.91059446847,-426966.5360245444,160.81447676174065,0.9612384510936428
150,0.3000000000000002,-474248.99144321284,55470.274314481765,-418778.7171287311,175.66835063875286,0.9620367432747798
200,0.4000000000000003,-470619.1630287387,59269.71896595234,-411349.44406278635,187.7007803232193,0.962036743274

2017-02-20 14:14:56 1487625296.22 | 91.39 CPU sec: RestrEquil: finishing MD run


RestrEquil: system energy  after MD: -101686.97918387999 kcal/mol


In [18]:
equilPositions = equilState.getPositions()
print(initPositions[0])
print(equilPositions[0])

(33.43, 11.279999999999999, 44.490000000000009) A
(3.3973052501678467, 1.1316759586334229, 4.442593097686768) nm


In [19]:
equilVelocities = equilState.getVelocities()
print(equilVelocities[:10])

[(-0.5905418992042542, 0.2360309511423111, -0.7270171642303467), (-0.3419838547706604, 0.3494241535663605, 0.19692721962928772), (-0.7442020177841187, 1.0486479997634888, -0.20452308654785156), (0.018204649910330772, 0.043820444494485855, 0.15904010832309723), (0.13965833187103271, 0.6059970259666443, -0.7374553680419922), (-0.9233154058456421, -0.5337115526199341, -0.17551156878471375), (-0.35443344712257385, -0.0903269574046135, 0.16326820850372314), (0.36805152893066406, -0.34695589542388916, 0.5089651942253113), (-1.3681297302246094, 1.4506207704544067, -1.9237070083618164), (1.6266942024230957, -1.6523851156234741, 2.9657161235809326)] nm/ps
